# Monitoramento de Rotas e Latência com TraceRoute e Ping
## Redes de Computadores
### Alunos: Leonardo de Oliveira Nanes, Gabriel Lázaro e Gabriel Calabrese 

Nesta apresentação, vamos explorar o conceito de **TraceRoute** com **Ping** imbutido para monitoramento de redes e visualização de hops, latência e perda de pacotes entre o host e o destino. Além disso, plotaremos as rotas geograficamente em um mapa interativo.

---

### Estrutura da apresentação:
-  Introdução aos conceitos de TraceRoute e Ping.
-  Implementação prática das ferramentas.
-  Visualização das rotas no mapa.

---

## O que é TraceRoute?

O **TraceRoute** é uma ferramenta de diagnóstico de redes que rastreia o caminho percorrido pelos pacotes de dados entre a origem (host) e o destino. Ele identifica os roteadores intermediários (hops) que os pacotes atravessam até chegar ao seu destino final.

- **Funcionamento**: O TraceRoute envia pacotes com valores incrementais de TTL (Time to Live). Quando o TTL chega a 0, o roteador emite uma mensagem ICMP "time exceeded", revelando sua presença. Esse processo se repete até alcançar o destino final ou até o TTL máximo.

---

## O que é Ping?

O **Ping** é uma ferramenta usada para testar a conectividade entre dois dispositivos em uma rede. Ele mede a latência (tempo de resposta) e a perda de pacotes. O Ping utiliza pacotes ICMP "Echo Request" e "Echo Reply" para testar se um dispositivo está acessível e para obter informações sobre o tempo de resposta.

- **Latência**: Tempo que um pacote leva para ir até o destino e voltar.
- **Perda de Pacotes**: Proporção de pacotes enviados que não receberam resposta.


# IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import socket
import threading
import requests
import ipaddress
from folium.plugins import TimestampedGeoJson
from datetime import datetime, timedelta
from numpy import random
import folium
import pingparsing
import webbrowser
import tkinter as tk
from tkinter import ttk
import pandas as pd


## Funções: TraceRoute, Ping e Geolocalização

Aqui estão as funções principais para realizar o TraceRoute com Ping e para obter as coordenadas dos IPs através de uma API pública.

- `traceroute_with_ping`: Realiza o TraceRoute até o destino especificado, verificando a latência e perda de pacotes a cada hop.
- `get_coordinates`: Obtém as coordenadas geográficas do IP usando uma API para plotagem de roteadores no mapa.
- `is_private_ip`: Verifica se o IP é privado e, assim, não acessível por APIs públicas.


# Funções auxiliares

In [2]:
def atualizar_dataframe(tree, df):
    for item in tree.get_children():
        tree.delete(item)
    for index, row in df.iterrows():
        tree.insert("", "end", values=row.tolist())

def iniciar_traceroute(tree):
    destino = entry_destino.get()  # Obter o IP ou host da entrada
    max_hops = int(entrada_max_hops.get())  # Obter o valor máximo de hops
    timeout = int(entrada_timeout.get())  # Obter o timeout
    ping_count = int(entrada_ping_count.get())  # Obter a contagem de pings
    
    if destino:
        threading.Thread(target=traceroute_with_ping, args=(tree, destino, max_hops, timeout, ping_count)).start()


# TRACEROUTE COM PING

In [3]:
def traceroute_with_ping(tree, host, max_hops=30, timeout=2, ping_count=4):
    df = pd.DataFrame(columns=["HOP", "IP", "Packets Sent", "Packet Loss",
                               "Avg Time (ms)", "Min Time (ms)", "Max Time (ms)", "Std Dev (ms)"])
    try:
        dest_addr = socket.gethostbyname(host)
    except socket.gaierror:
        print(f"Erro: não foi possível resolver o host '{host}'")
        return

    port, icmp, udp = 33434, socket.getprotobyname('icmp'), socket.getprotobyname('udp')
    ping_parser, transmitter = pingparsing.PingParsing(), pingparsing.PingTransmitter()
    send_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, udp)

    hops = []  # Para armazenar os endereços IP dos hops
    executando = True  # Controle de execução

    for ttl in range(1, max_hops + 1):
        if not executando:
            break  # Para caso a janela seja fechada

        recv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, icmp)
        send_socket.setsockopt(socket.SOL_IP, socket.IP_TTL, ttl)
        recv_socket.settimeout(timeout)
        recv_socket.bind(("", port))

        curr_addr = None
        new_row = {"HOP": ttl, "IP": "*", "Packets Sent": "", "Packet Loss": "",
                   "Avg Time (ms)": "Timeout", "Min Time (ms)": "", "Max Time (ms)": "", "Std Dev (ms)": ""}

        try:
            send_socket.sendto(b"", (host, port))
            _, curr_addr = recv_socket.recvfrom(512)
            curr_addr = curr_addr[0]
            hops.append(curr_addr)  # Armazenando o IP do hop

            # Enviando o ping
            transmitter.destination = curr_addr
            transmitter.count = ping_count
            ping_result = transmitter.ping()

            if ping_result.returncode == 0:
                ping_stats = ping_parser.parse(ping_result.stdout)
                new_row.update({
                    "IP": curr_addr,
                    "Packets Sent": ping_stats.packet_transmit,
                    "Packet Loss": ping_stats.packet_loss_rate,
                    "Avg Time (ms)": ping_stats.rtt_avg,
                    "Min Time (ms)": ping_stats.rtt_min,
                    "Max Time (ms)": ping_stats.rtt_max,
                    "Std Dev (ms)": ping_stats.rtt_mdev
                })
        except socket.timeout:
            pass
        except socket.error as e:
            print(f"{ttl}: erro no socket - {e}")

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        atualizar_dataframe(tree, df)
        tree.master.update_idletasks()

        recv_socket.close()

        if curr_addr == dest_addr:
            break

    send_socket.close()
    plot_route(hops)
    return hops

# MAPA

## Vendo se o IP é privado

In [4]:
def is_private_ip(ip):
    return ipaddress.ip_address(ip).is_private

## Pegando as coordenadas

In [5]:
def get_coordinates(ip):
    if is_private_ip(ip):
        print(f"IP {ip} é privado. Sem coordenadas públicas conhecidas.")
        return None

    try:
        response = requests.get(f"http://ip-api.com/json/{ip}")
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'success':
                lat, lon = data['lat'], data['lon']
                # Exibe as coordenadas no console
                print(f"Coordenadas de {ip}: ({lat}, {lon})")
                return lat, lon
            else:
                print(f"API retornou falha para o IP {ip}: {data['message']}")
                return None
        else:
            print(
                f"Erro ao conectar-se à API para o IP {ip}. Status HTTP: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Erro de conexão ao tentar obter localização de {ip}: {e}")
        return None

# PLOTANDO O MAPA

In [6]:
from folium.plugins import TimestampedGeoJson
from datetime import datetime,timedelta
from numpy import random

def jitter_coordinates(coords):
    jitter = 0.02  # Pequeno deslocamento
    return (coords[0] + random.uniform(-jitter, jitter), coords[1] + random.uniform(-jitter, jitter))


def plot_route(hops):
    linhas = []
    data_inicial = datetime(2024, 1, 1, 0, 0, 0)  # Data inicial arbitrária
    intervalo_tempo = timedelta(minutes=1)
    seen_coords = set()

    def inverter_coordenada(coordenada):
        latitude, longitude = coordenada
        return longitude, latitude

    for i, hop in enumerate(hops):
        coords = get_coordinates(hop)
        if coords:  # Verifica se coords não é None
            coords = inverter_coordenada(coords)
            if coords in seen_coords:
                coords = jitter_coordinates(coords)
                seen_coords.add(coords)
            data_atual = data_inicial + i * intervalo_tempo
            data_formatada = data_atual.isoformat() + "Z"
            linhas.append({"coord": coords, "datas": data_formatada})
        else:
            print(f"Coordenadas não encontradas para o hop {hop}. Saltando este hop.")

    features = []
    for i in range(len(linhas) - 1):  # Itera pelas linhas criando as conexões
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [linhas[i]["coord"], linhas[i + 1]["coord"]],
                "icon": "circle",
                "iconstyle": {"fillColor": "#16BAB6", "fillOpacity": 0.5, "radius": 2}
            },
            "properties": {
                "times": [linhas[i]["datas"], linhas[i + 1]["datas"]],
                "style": {"color": "#16BAB6", "weight": 1},
            }
        }
        features.append(feature)

    # Criar o mapa com o ponto inicial sendo o primeiro hop
    if linhas:
        inicial_latlon = inverter_coordenada(linhas[0]["coord"])
        mapa = folium.Map(location=inicial_latlon, tiles="OpenStreetMap", zoom_start=5)
        for i in range(len(linhas)):
            folium.Marker(
                location=inverter_coordenada(linhas[i]["coord"]),
                popup=f"Hop {i + 1}: Coordenadas: {inverter_coordenada(linhas[i]['coord'])}"
            ).add_to(mapa)

        # Adicionar os dados de rota animados ao mapa
        TimestampedGeoJson(
            data={"type": "FeatureCollection", "features": features},
            period="PT1M",  # Tempo entre os "frames" da animação
            add_last_point=True,  # Adiciona o último ponto
            loop=False,
            auto_play=True  
        ).add_to(mapa)

        # Salvar o mapa como um arquivo HTML
        mapa_file_path = "mapa.html"
        mapa.save(mapa_file_path)

        # Abrir o mapa no navegador
        webbrowser.open(mapa_file_path)

        return mapa
    else:
        print("Nenhuma coordenada válida encontrada para exibir no mapa.")
        return None



# Interface Gráfica

In [7]:
def ao_fechar_janela():
    global executando
    executando = False  # Para parar a execução do traceroute
    root.destroy()  # Fecha a janela principal

def criar_interface():
    global tree, entry_destino, entrada_max_hops, entrada_timeout, entrada_ping_count, root  # Tornar as variáveis globais

    root = tk.Tk()  # Cria a janela principal
    root.title("Traceroute")
    root.protocol("WM_DELETE_WINDOW", ao_fechar_janela)  # Define a função a ser chamada ao fechar a janela

    # Frame para entrada de dados
    frame_input = tk.Frame(root)
    frame_input.pack(pady=10)

    # Campo de entrada para IP ou host
    label_destino = tk.Label(frame_input, text="Digite o IP ou endereço do host:")
    label_destino.pack(side=tk.LEFT, padx=5)

    entry_destino = tk.Entry(frame_input, width=30)
    entry_destino.pack(side=tk.LEFT, padx=5)

    # Botão para iniciar o traceroute
    botao_iniciar = tk.Button(frame_input, text="Iniciar Traceroute", command=lambda: iniciar_traceroute(tree))
    botao_iniciar.pack(side=tk.LEFT, padx=5)

    # Frame para as configurações adicionais
    frame_config = tk.Frame(root)
    frame_config.pack(pady=5)

    # Campo de entrada para Max Hops
    label_max_hops = tk.Label(frame_config, text="Máximo de Hops:")
    label_max_hops.pack(side=tk.LEFT, padx=5)
    entrada_max_hops = tk.Entry(frame_config, width=5)
    entrada_max_hops.insert(0, "30")  # Valor padrão
    entrada_max_hops.pack(side=tk.LEFT, padx=5)

    # Campo de entrada para Timeout
    label_timeout = tk.Label(frame_config, text="Timeout (segundos):")
    label_timeout.pack(side=tk.LEFT, padx=5)
    entrada_timeout = tk.Entry(frame_config, width=5)
    entrada_timeout.insert(0, "2")  # Valor padrão
    entrada_timeout.pack(side=tk.LEFT, padx=5)

    # Campo de entrada para Ping Count
    label_ping_count = tk.Label(frame_config, text="Contagem de Pings:")
    label_ping_count.pack(side=tk.LEFT, padx=5)
    entrada_ping_count = tk.Entry(frame_config, width=5)
    entrada_ping_count.insert(0, "4")  # Valor padrão
    entrada_ping_count.pack(side=tk.LEFT, padx=5)

    # Criação da Treeview para mostrar resultados
    tree = ttk.Treeview(root, columns=("HOP", "IP", "Packets Sent", "Packet Loss",
                                       "Avg Time (ms)", "Min Time (ms)", "Max Time (ms)", "Std Dev (ms)"), show="headings")
    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, width=100, anchor= 'center')
    style = ttk.Style()
    style.configure("Treeview", font=("Arial", 12))
    tree.pack(pady=10)
    tree.pack(pady=10, fill=tk.BOTH, expand=True)
    root.mainloop()  # Inicia o loop principal da interface


## Execução do Programa



In [ ]:
if __name__ == "__main__":
    criar_interface()